ENUNCIADO c) La cadena de supermercados "La Mocha" está evaluando crear tarjetas con beneficios para las personas que compran en ella, por lo que se disponen a agrupar a estas personas dependiendo de el ingreso que tienen y la cantidad que gastan, a fin de generar dos o tres grupos a los cuales se les va a asignar una tarjeta con ciertos beneficios.
Detallá los pasos a seguir para que, a partir del data frame curado (sin las anomalías) se obtengan los grupos requeridos y cómo evalúa si lo realizado es correcto. AVISO: para este punto no hay que ejecutar código, sino detallar los pasos.

tienen que crear tarjetas de beneficios para los q compran en supermercado, disponen de datos(ingresos, compras mensuales en un pago, etc)
hay que agrupar clientes para ver su omportamiento y decidir cuantas tarjetas hacer y de que tipos
ej aux: clientes que gastan mucho y gastan mucho ->un tipo de beneficio y para clientes que ganan poco y gastan poco -> otro tipo de beneficio
tenemos que definir como agrupar clientes para definir que tarjeta darle a cada uno

pasos a partir de tener un df al que ya le limpiamos todas las anomalias:
tenemos que hacer los pasos que fuimos viendo en "clustering"


PASO 1:
decidir que datos vamos a utilizar(elegir columnas de nuestro df) para realizar este agrupamiento:
vamos a utilizar ingresos de una persona respecto de pagos mensuales en 1 pago(aux:tmbn podria ser en cuotas, etc) que hace una persona ->agrupo personas con ese criterio (en este caso lo q no podria hacer es utilizar ingreso respecto de ahorro, dado que eso no daria ninguna informacion al supermercado)

PASO 2:
ver como se comportan estos 2 tipos de datos (distribucion)->ver graficamente como se comporta el ingreso y como se comporta el pago en una sola cuota 
#para ello hacer grafico:
import pandas as pd
import seaborn as sns
datos=pd.read_csv("data.csv") #cargo datos en un df
sns.pairplot(datos)  #hago grafico y alli presto atencion a los graficos ingreso y comportamiento de pago en una cuota

PASO 3:
hago grafico de codo para ver masomenos cuantos grupos debo hacer y ver donde se quiebra la curva, en que grupo se quiebra y a partir de alli decidir que cantidad de grupos utilizar
#Voy calculando kmeans de cada grupo:
inercias={}
def inercias_por_k(diccionario):
    for i in range(1,11):
        kmeans=KMeans(n_clusters=i, init="random", n_init=10, max_iter=300, random_state=123457)
        kmeans.fit(datos_escaleado)
        diccionario[i]=kmeans.inertia_

inercias_por_k(inercias)

inercias

#me devuelve:En diccio para cada grupo su valor de inercia aux: Grafico valor de inercia por cantidad de grupos que tenemos

#Auxiliar:
df_inercias=pd.DataFrame(inercias.items(), columns=["k", "inercia"])
df_inercias

#Grafico codo -> el  mas importante para saber bien interpretar inercias
sns.pointplot(data=df_inercias, x="k", y="inercia") 


PASO 4:
a partir de grafico de codo, decidir cantidad  de grupos 

PASO 5:
una vez decidida la cantidad de grupos, hago el clustering con esa cantidad de grupos ->kmean=cant de grupos (ej:si son 4 grupos ->kmeans=4)
#muestra en general:
k = 4  #definimos la cantidad de clusters  ->aux:en caso de ser 4 grupos
kmeans = KMeans(n_clusters = k, init="random", n_init=10, max_iter=300, random_state=123457) #tomamos los centroides de forma aleatoria y definimos un máximo de 300 iteraciones
kmeans.fit(datos_escaleado)  #aplicamos el método a nuestros datos


#auxiliar: si quiero mostrar especificamente algunas columnas ->en codigo de abajo poner -> datos_escaleado[posicion de columna que quiero mostrar]
import seaborn as sns
colores = ["red", "green", "blue"]#Colores
g=sns.set(rc={"figure.figsize":(11.7,8.27)}) #Tamaño (esto lo agrego guille)
g = sns.scatterplot(x = datos_escaleado[:,2], y = datos_escaleado[:,3], hue = kmeans.labels_, palette = colores, alpha = 0.5) #Grafica puntos
g = sns.scatterplot(x = kmeans.cluster_centers_[:,2], y = kmeans.cluster_centers_[:,3], zorder = 10, palette = colores, hue = [0, 1, 2], legend = False, marker=6, s=200) 
#Pone centroides en el gráfico (traingulos que muestran las distancias)

#auxiliar:para saber que significa el[:,nro]-> iris_escaleado[:,2]#para obtener datos de una columna ->lista completa de principio a fin pero q solo tome valores (de todos los datos solo pide la posicion 2 de casa lista por eso el “:,”)

PASO 6:
hago grafico silhouette -> para ver si datos estan bien distribuidos los puntos:
#grafico nos dice si nuestros datos son confiables o no 
from sklearn.metrics import silhouette_samples, silhouette_score #Para el coeficiente de silhouette
silhouette_avg = silhouette_score(datos_escaleado, kmeans.labels_) #Calculamos el promedio del silhouette de todos
sample_silhouette_values = silhouette_samples(datos_escaleado, kmeans.labels_) #Calculamos el silhouette de cada punto

#Para generar el gráfico:
import matplotlib.pyplot as plt 
import matplotlib.cm as cm 
import numpy as np
def graficarSilhouette (k, labels, sample_silhouette_values, silhouette_avg):
  fig, ax1 = plt.subplots(1, 1)
  y_lower = 10
  for i in range(k):
      ith_cluster_silhouette_values = \
          sample_silhouette_values[labels == i]

      ith_cluster_silhouette_values.sort()

      size_cluster_i = ith_cluster_silhouette_values.shape[0]
      y_upper = y_lower + size_cluster_i

      color = cm.nipy_spectral(float(i) / k)
      ax1.fill_betweenx(np.arange(y_lower, y_upper),
                        0, ith_cluster_silhouette_values,
                        facecolor=color, edgecolor=color, alpha=0.7)
      ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
      y_lower = y_upper + 10

  ax1.set_title("Plot del silhouette de cada cluster")
  ax1.set_xlabel("Coeficiente de silhouette")
  ax1.set_ylabel("Etiqueta del cluster")
  ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
  ax1.set_yticks([]) 

#Para visualizar el grafico:
graficarSilhouette (k, kmeans.labels_, sample_silhouette_values, silhouette_avg) #aca voy a ver los 3 grupos (el 0, 1 y 2 ) me tengo que ir fijando si esta + cerca de 1 o -1








ENUNCIADO d) Dados los gráficos ([Codo1](https://github.com/FundamentosInformaticaUCEMA/Simulacro_P2_2022/blob/master/Codo1.png) y [Silhouette1](https://github.com/FundamentosInformaticaUCEMA/Simulacro_P2_2022/blob/master/Silhouette1.png) por un lado, y [Codo2](https://github.com/FundamentosInformaticaUCEMA/Simulacro_P2_2022/blob/master/Codo2.png) y [Silhouette2](https://github.com/FundamentosInformaticaUCEMA/Simulacro_P2_2022/blob/master/Silhouette2.png) por otro), analizar y evaluar si el proceso de clustering fue correcto en cada caso. Explicitar tanto lo que está bien como lo que considerás que está mal.

codo1:
se ve perfectamente donde esta el quiebre del grafico ->viene teniendo una misma pendiente y de repente cambia completamente cuando k=3 (alli es el quiebre, es decir pasa del brazo al entebrazo)-> como el quiebre se da en k=3 ->tengo 3 grupos)(alli tengo la menor cantidad de inercia y la menor cantidad de grupos al mismo tiempo) ->cantidad de grupos que tiene son 3

silhouette 1:
me muestra los 3 grupos(0, 1 y 2), veo que no hay datos que sean negativos o tiendan a valores bajos, casi todos tienden a 1. en el grupo 1 veo que hay datos que me quedan medio cortos. linea roja punteada->idealmente ptos deben pasar esa linea

En su gran mayoria se cumple que estos datos se pueden agrupar bien en 3 grupos.

codo 2:
cambio de pendiente no se entiende-> es xq datos son malos y no sirven para agruparse
ningun nro de grupos esta bien, xq no hay un codo

silhouette 2:
valor max que alcanzan es 0.4 ->es muy bajo
hay grupos que tienden a nros negativos
nro negativos dicen que ->no me da la certeza que datos que aparecen corresponden al grupo que alli aparece 




datos aux de graficos:
aux:
codo:
calcular cuan compactos son los grupos obtenidos(funcionó mejor si todos los elementos del grupo están lo más cerca posible de su centro->para ello sumar las distancias de cada punto a su respectivo centro y usar eso como medida)->a este valor se lo denomina inercia
#Inercia: valor q dice q tan confiable es nuestro valor de grupos (es mejor buscar inercias bajas xq son mas confiables)
solhouette:
grafico nos dice si nuestros datos son confiables o no ->Mas cercanos a 1 son mas confiables(los grupos se separaron bien es q es confiable)  y mas cercanos a -1 son menos confiables ->muestra los 3 grupos en este caso y me fijo si cada uno esta mas cerca o mas lejos de 1 y -1 